## Period luminosity relations
We analyse the PL relation on a set of 300 Miras and 300 SRVs from LMC. Same for SMC

In [ ]:
import numpy as np
from astropy.table import Table
from matplotlib import pyplot as plt
from scipy import optimize as opt
import os

currentPath = os.getcwd()
reportPath = "../Rapport/Images/Graphs/PLRs"

fSize = 13

#### Function funcpoly :  
Evaluates a polynomial function of the type
\begin{equation}
f(x) = a_0 + a_1x + a_2x^2 + ... + a_nx^n
\end{equation}

Parameters :  
- x  : The value to evaluate the function  
- *a : Tuple containing the coefficients $(a_0, a_1, ..., a_n)$

In [ ]:
def funcPoly(x, *a):
    return sum([a[i]*x**i for i in range(len(a))])

We load the data from the crossmatch made on TOPCAT. we create the variables lmc and smc. For each variables, we create a new entry $W_jk$ to prevent some unprecision due to the distance. We also create the entry "C" containing booleans if the star is an C-rich.

In [ ]:
# Charger les données
dirData = "../TOPCAT/"
pathLMC = dirData + "OGLE_LMC_2MASS_ALLWISE_GAIAEDR3_RESULTS_CrossMatch.fits"
pathSMC = dirData + "OGLE_SMC_2MASS_ALLWISE_GAIAEDR3_RESULTS_CrossMatch.fits"
# Load the data
lmc = Table.read(pathLMC, format="fits")
lmc["Wjk"] = lmc["Kmag_2MASS"] - 0.686 * ( lmc["Jmag_2MASS"] - lmc["Kmag_2MASS"] )
lmc["C"] = lmc["Spectr"] == "C-rich"

smc = Table.read(pathSMC, format="fits")
smc["Wjk"] = smc["Kmag_2MASS"] - 0.686 * ( smc["Jmag_2MASS"] - smc["Kmag_2MASS"] )
smc["C"] = smc["Spectr"] == "C-rich"

In [ ]:
#sel = [True if type(_) != type(np.ma.core.MaskedConstant()) else False for _ in lmc['P1'] ]

#stars = lmc['Type'][sel]
#print(stars)

In [ ]:
#print(len(lmc['P1'][~np.isnan(lmc['P1'])]))
#print(set(lmc['Type'][~np.isnan(lmc['P1'])]))
#print(len(lmc['PFM'][~np.isnan(lmc['PFM']) & (lmc['Type']=='SRV')]))
#print(set(lmc['Type'][~np.isnan(lmc['PFM'])]))

In [ ]:
# Graphique de Wjk vs log(P1)
xLim = (1.5, 3.7)
yLim = (13.5, 2.5)

# y = a_0 + a_1*x  sepLines = [[a_0, a_1], ...]
lmcSepLines = [[21, -5], [23.3, -5]]

x = np.log10(lmc["P1"])
y = lmc["Wjk"]
sel = lmc["C"] # selection

xx = np.linspace(*xLim, 2)

fig = plt.figure(figsize=(6, 4), dpi=150)
ax = fig.add_subplot()
ax.set_xlabel("log(P1)", fontsize=fSize)
ax.set_ylabel("Wjk", fontsize=fSize)
ax.set_xlim(xLim)
ax.set_ylim(yLim)
[ax.scatter(x[_], y[_], c=c, label=l, s=1) for _, c, l in zip([sel, ~sel], "rb", ["C-rich", "O-rich"])]
[ax.plot(xx, funcPoly(xx, *_), c=c, linewidth=0.9) for _, c in zip(lmcSepLines, ["gray", "gray"])]
ax.legend(fontsize=8)

In [ ]:
# Primary period is not always the fundamental mode
seqC1 = (lmc["Wjk"] > funcPoly(np.log10(lmc["P1"]), *lmcSepLines[0])) & ( (lmc["Wjk"] < funcPoly(np.log10(lmc["P1"]), *lmcSepLines[1])) ) 
seqC2 = (lmc["Wjk"] > funcPoly(np.log10(lmc["P2"]), *lmcSepLines[0])) & ( (lmc["Wjk"] < funcPoly(np.log10(lmc["P2"]), *lmcSepLines[1])) ) 
seqC3 = (lmc["Wjk"] > funcPoly(np.log10(lmc["P3"]), *lmcSepLines[0])) & ( (lmc["Wjk"] < funcPoly(np.log10(lmc["P3"]), *lmcSepLines[1])) ) 
PFM = 1*lmc["P1"] # Period of the fundamental mode

sel = ~seqC1 & seqC2
PFM[sel] = lmc["P2"][sel]

sel = ~seqC1 & ~seqC2 & seqC3
PFM[sel] = lmc["P3"][sel]

sel = ~seqC1 & ~seqC2 & ~seqC3
PFM[sel] = np.nan

lmc["PFM"] = 1*PFM

In [ ]:
xLim = (1.5, 3.7)
yLim = (13.5, 2.5)

x = np.log10(lmc["P1"])
y = lmc["Wjk"]
sel = lmc["C"]
xx = np.linspace(*xLim, 2)

fig = plt.figure(figsize=(6, 4), dpi=300)
ax = fig.add_subplot()
ax.set_xlim(xLim)
ax.set_ylim(yLim)
ax.set_xlabel("log(P1)", fontsize=fSize)
ax.set_ylabel("$W_{jk}$", fontsize=fSize)
ax.scatter(x, y, s=1)
[ax.scatter(x[seqC1&_], y[seqC1&_], c=c, s=1, label=l) for _, c, l in zip([sel, ~sel], "rb", ["C-rich", "O-rich"])]
[ax.plot(xx, funcPoly(xx, *_), c="gray", linewidth=0.9) for _ in lmcSepLines]
ax.legend(fontsize=8)
#fig.savefig("{0}/Output/PLRs/{1}".format(currentPath, "PLR_LMC_tot.png"), bbox_inches = "tight")
#fig.savefig("{0}/{1}".format(reportPath, "PLR_LMC_tot.png"), bbox_inches = "tight")

In [ ]:
xLim = (1.5, 3.7)
yLim = (13.5, 2.5)

x = np.log10(PFM)
y = lmc["Wjk"]
_s = lmc["C"]


xx = np.linspace(*xLim, 2)

fig = plt.figure(figsize=(6, 4), dpi=300)
ax = fig.add_subplot()
ax.set_xlim(xLim)
ax.set_ylim(yLim)
ax.set_xlabel("log(P1)")
ax.set_ylabel("$W_{jk}$")

for s, c, l in zip([_s, ~_s], "rb", ["C-rich", "O-rich"]):

    ax.scatter(x[s], y[s], c=c, s=1, label=l)
    p0 = [1, 1]

    sel = ~np.isnan(x) & ~np.isnan(x)

    par, cov = opt.curve_fit(funcPoly, x[sel & s], y[sel & s], p0)
    err = np.sqrt(np.diag(cov))
    print('{} par : {}'.format(c, par))
    print('{} err : {}'.format(c, err))
    ax.plot(xx, funcPoly(xx, *par), c=c, linewidth=0.9, label="{0:.1f} $\pm$ {1:.1f} + ({2:.1f} $\pm$ {3:.1f})x".format(par[0], err[1], par[1], err[1]))
    
ax.legend(fontsize=8)

#fig.savefig("{0}/Output/PLRs/{1}".format(currentPath, "PLR_LMC_lin.png"), bbox_inches = "tight")
#fig.savefig("{0}/{1}".format(reportPath, "PLR_LMC_lin.png"), bbox_inches = "tight")

qLMC = par[0]
print('qLMC : {}'.format(qLMC))

SAME THING FOR THE SMC

In [ ]:
xLim = (1.5, 3.7)
yLim = (13.5, 2.5)

# y = a_0 + a_1*x  sepLines = [[a_0, a_1], ...]
smcSepLines = [[21.3, -5], [23.6, -5]]

x = np.log10(smc["P1"])
y = smc["Wjk"]
s = smc["C"] # selection

xx = np.linspace(*xLim, 2)

fig = plt.figure(figsize=(6, 4), dpi=300)
ax = fig.add_subplot()
ax.set_xlabel("log(P1)")
ax.set_ylabel("$W_{jk}$")
ax.set_xlim(xLim)
ax.set_ylim(yLim)
[ax.scatter(x[_], y[_], c=c, s=1) for _, c in zip([s, ~s], "rb")]
[ax.plot(xx, funcPoly(xx, *_), c='gray', linewidth=0.9) for _ in smcSepLines]

In [ ]:
# Primary period is not always the fundamental mode

seqC1 = (smc["Wjk"] > funcPoly(np.log10(smc["P1"]), *smcSepLines[0])) & ( (smc["Wjk"] < funcPoly(np.log10(smc["P1"]), *smcSepLines[1])) ) 
seqC2 = (smc["Wjk"] > funcPoly(np.log10(smc["P2"]), *smcSepLines[0])) & ( (smc["Wjk"] < funcPoly(np.log10(smc["P2"]), *smcSepLines[1])) ) 
seqC3 = (smc["Wjk"] > funcPoly(np.log10(smc["P3"]), *smcSepLines[0])) & ( (smc["Wjk"] < funcPoly(np.log10(smc["P3"]), *smcSepLines[1])) ) 
PFM = 1*smc["P1"] # Period of the fundamental mode

sel = ~seqC1 & seqC2
PFM[sel] = smc["P2"][sel]

sel = ~seqC1 & ~seqC2 & seqC3
PFM[sel] = smc["P3"][sel]

sel = ~seqC1 & ~seqC2 & ~seqC3
PFM[sel] = np.nan

smc["PFM"] = 1*PFM

In [ ]:
xLim = (1.5, 3.7)
yLim = (13.5, 2.5)

x = np.log10(smc["P1"])
y = smc["Wjk"]
sel = smc["C"]

xx = np.linspace(*xLim, 2)

fig = plt.figure(figsize=(6, 4), dpi=300)
ax = fig.add_subplot()
ax.set_xlim(xLim)
ax.set_ylim(yLim)
ax.set_xlabel("log(P1)")
ax.set_ylabel("$W_{jk}$")
ax.scatter(x, y, s=1)
[ax.scatter(x[seqC1&_], y[seqC1&_], c=c, s=1, label=l) for _, c, l in zip([sel, ~sel], "rb", ["C-rich", "O-rich"])]
[ax.plot(xx, funcPoly(xx, *_), color="gray", linewidth=0.9) for _ in smcSepLines]

ax.legend(fontsize=8)

#fig.savefig("{0}/Output/PLRs/{1}".format(currentPath, "PLR_SMC_tot.png"), bbox_inches = "tight")
#fig.savefig("{0}/{1}".format(reportPath, "PLR_SMC_tot.png"), bbox_inches = "tight")

In [ ]:
xLim = (1.5, 3.7)
yLim = (13.5, 2.5)

x = np.log10(PFM)
y = smc["Wjk"]
_s = smc["C"]

xx = np.linspace(*xLim, 2)

fig = plt.figure(figsize=(6, 4), dpi=300)
ax = fig.add_subplot()
ax.set_xlim(xLim)
ax.set_ylim(yLim)
ax.set_xlabel("log(P1)")
ax.set_ylabel("$W_{jk}$")

for s, c, l in zip([_s, ~_s], "rb", ["C-rich", "O-rich"]):

    ax.scatter(x[s], y[s], c=c, s=1, label=l)
    p0 = [1, 1]

    sel = ~np.isnan(x) & ~np.isnan(x)

    par, cov = opt.curve_fit(funcPoly, x[sel & s], y[sel & s], p0)
    err = np.sqrt(np.diag(cov))
    print(par)
    ax.plot(xx, funcPoly(xx, *par), c=c, linewidth=0.9, label="{0:.1f} $\pm$ {1:.1f} + ({2:.1f} $\pm$ {3:.1f})x".format(par[0], err[1], par[1], err[1]))

ax.legend(fontsize=8)    

#fig.savefig("{0}/Output/PLRs/{1}".format(currentPath, "PLR_SMC_lin.png"), bbox_inches = "tight")
#fig.savefig("{0}/{1}".format(reportPath, "PLR_SMC_lin.png"), bbox_inches = "tight")

In [ ]:
def fPLR(x, q): # Period Luminosity Relation
    """
    x : log(p)
    q : intercept
    """
    return q - 3.89732574 * x

In [ ]:
xLim = (1.5, 3.7)
yLim = (13.5, 2.5)

x = np.log10(PFM)
y = smc["Wjk"]
s = ~smc["C"]

xx = np.linspace(*xLim, 2)

fig = plt.figure(figsize=(6, 4), dpi=300)
ax = fig.add_subplot()
ax.set_xlim(xLim)
ax.set_ylim(yLim)
ax.set_xlabel("log(P1)")
ax.set_ylabel("$W_{jk}$")

c='b'

ax.scatter(x[s], y[s], c=c, s=1, label="O-rich")
p0 = [1]

sel = ~np.isnan(x) & ~np.isnan(x)

par, cov = opt.curve_fit(fPLR, x[sel & s], y[sel & s], p0)
print(par)
qSMC = par[0]
ax.plot(xx, fPLR(xx, *par), c=c, linewidth=0.9, label="{0:.1f} $\pm$ {1:.1f} + ({2:.1f} $\pm$ {3:.1f})x".format(par[0], err[0], -3.9, 0.1))
ax.legend(fontsize=8)

#fig.savefig("{0}/Output/PLRs/{1}".format(currentPath, "PLR_SMC_linFinal.png"), bbox_inches = "tight")
#fig.savefig("{0}/{1}".format(reportPath, "PLR_SMC_linFinal.png"), bbox_inches = "tight")

In [ ]:
delta_q = qLMC - qSMC
muLMCTheo = 18.49
muSMCTheo = 18.96
muSMC = muLMCTheo - delta_q

print("delta_q = {0} - {1} = {2}".format(qLMC, qSMC, delta_q))

print("mu_SMC = {0}".format(muSMC))

ERROR estimation

In [ ]:
deltabLMC = 0.1 
deltabSMC = 0.3
muLMCTheo = 18.49
deltaMuLMC = 0.09
deltaMu = deltabLMC**2 + deltabSMC**2 + deltaMuLMC**2

print('Delta  mu scm = {:.1f}'.format(np.sqrt(deltaMu)))